In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import re #regex

Eventually create a list of major codes and run a for loop to get all dataframes

In [9]:
def extract_course_catalog_info(dept): #where dept is a string
    site_url = 'https://www.ucsd.edu/catalog/courses/' + dept + '.html'
    page = requests.get(site_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    

In [10]:
#site_url = 'https://www.ucsd.edu/catalog/courses/' + 'dept' + '.html'

dept = 'MATH'
site_url = 'https://www.ucsd.edu/catalog/courses/MATH.html'
page = requests.get(site_url)
soup = BeautifulSoup(page.content, 'html.parser')

#https://stackoverflow.com/questions/49010813/how-to-find-a-particular-class-in-beautifulsoup-using-findall

# Looking at just names and descriptions w/ htmlclass separation

In [100]:
#full_list = soup.find_all("p", {"class": "anchor-parent"}) 

names = soup.find_all("p", {"class": "course-name"})
descriptions = soup.find_all("p", {"class": "course-descriptions"}) #includes prereqs (so want to extract that)

In [145]:
df = pd.DataFrame(columns=['Course Name', 'Course Description', 'Course Units'])
for course, description in zip(names, descriptions):
    course, description = course.getText(), description.getText()
    
    # Course ID check to filter upper div. classes.
    course_id = course[course.index(' ')+1:course.index('.')]
    while course_id[-1].isalpha() or course_id[-1] == '-':
        course_id = course_id[:-1]
    course_id = int(course_id)
    if course_id >= 200:
        continue
    
    # Grabs course info.
    course_name = course[:course.index('.')]
    course_title = course[course.index('.')+1:course.index('(')-1]
    course_units = course[course.index('('):course.index(')')+1]
    
    # Append row to data frame.
    df = df.append({
        'Course Name': course_name,
        'Course Description': description,
        'Course Units': course_units
    }, ignore_index=True)
df

,Course Name,Course Description,Course Units
0,MATH 2,A highly adaptive course designed to build on ...,(4)
1,MATH 3C,Functions and their graphs. Linear and polynom...,(4)
2,MATH 4C,Review of polynomials. Graphing functions and ...,(4)
3,MATH 10A,Differential calculus of functions of one vari...,(4)
4,MATH 10B,Integral calculus of functions of one variable...,(4)
...,...,...,...
98,MATH 195,Students will be responsible for and teach a c...,(4)
99,MATH 196,A variety of topics and current research resul...,(1)
100,MATH 197,An enrichment program which provides work\n\t\...,(2 or 4)
101,MATH 199,Independent reading in advanced mathematics by...,(2 or 4)


In [44]:
print(names)

[<p class="course-name">MATH 2. Introduction to College Mathematics (4)</p>, <p class="course-name">MATH 3C. Precalculus (4)</p>, <p class="course-name">MATH 4C. Precalculus for Science and Engineering (4)</p>, <p class="course-name">MATH 10A. Calculus I (4)</p>, <p class="course-name">MATH 10B. Calculus II (4)</p>, <p class="course-name">MATH 10C. Calculus III (4)</p>, <p class="course-name">MATH 11. Calculus-Based Introductory Probability and Statistics (5)</p>, <p class="course-name">MATH 15A. Introduction to Discrete Mathematics (4)</p>, <p class="course-name">MATH 18. Linear Algebra (4)</p>, <p class="course-name">MATH 20A. Calculus for Science and Engineering (4)</p>, <p class="course-name">MATH 20B. Calculus for Science and Engineering (4)</p>, <p class="course-name">MATH 20C. Calculus
		  and Analytic Geometry for Science and Engineering (4)</p>, <p class="course-name">MATH 20D. Introduction to Differential Equations (4)</p>, <p class="course-name">MATH 20E. Vector Calculus (4)

In [12]:
name_list = [] #this is in HTML contains all course header info

for x in names:
    name_list.append(str(x))

In [26]:
course_ids = []
course_names = []
course_units = [] 

for n in np.arange(len(name_list) - 1):
    #for i in np.arange(len(name_list[n]) - 1):
    x = name_list[n]
    course_ids.append(x[len('<p class="course-name">') + len(dept) + 1 : x.index('.')])
    course_names.append(x[x.index('.')+2:len(x)-8])
    course_units.append(x[len(x)-6])

In [28]:
print(course_ids)
print(course_names)
print(course_units)

['2', '3C', '4C', '10A', '10B', '10C', '11', '15A', '18', '20A', '20B', '20C', '20D', '20E', '31AH', '31BH', '31CH', '87', '95', '96', '99R', '100A', '100B', '100C', '102', '103A', '103B', '104A', '104B', '104C', '105', '106', '109', '110', '111A', '111B', '112A', '112B', '114', '120A', '120B', '121A', '121B', '130', '140A', '140B', '140C', '142A', '142B', '144', '146', '148', '150A', '150B', '152', '153', '154', '155A', '155B', '157', '158', '160A', '160B', '163', '168A', '170A', '170B', '170C', '171A', '171B', '173A', '173B', '174', '175', '179', '180A', '180B', '180C', '181A', '181B', '181C', '181D', '181E', '181F', '183', '184', '185', '186', '187A', '187B', '188', '189', '190A', '190B', '191', '193A', '193B', '194', '195', '196', '197', '199', '199H', '200A-B-C', '201A', '202A', '202B', '202C', '203A', '203B', '203C', '204A', '204B', '204C', '205', '206A', '206B', '207A', '208', '209', '210A', '210B', '210C', '217', '218', '220A-B-C', '221A', '221B', '231A-B-C', '237A', '237B', '2

## Some course names are weird (why?) e.g. 'Foundations\n\t\t  of Teaching and Learning Mathematics I'

## Course units cut off (why?)

In [17]:
course_nums = []

for i in course_ids:
    course_nums.append(re.findall(r'\d+|\D+', i)) #regex makes id's in format 'Num' + 'Letter'    

#regex for this \d+|\D+

# just nums: r'^\d+'

#### use list comprehension it is cleaner

finding indices of class category

In [18]:


for i in course_nums:
    if len(i[0]) == 2:
        try:
            lower_div_i = course_ids.index(i[0]+i[1]) #last index of lower divisions
        except IndexError:
            lower_div_i = course_ids.index(i[0])
    if int(i[0]) < 200:
        try:
            upper_div_i = course_ids.index(i[0]+i[1]) #" " of upper divisions
        except IndexError:
            upper_div_i = course_ids.index(i[0])
        
#[i for i,val in enumerate(l) if val==]

https://stackoverflow.com/questions/6294179/how-to-find-all-occurrences-of-an-element-in-a-list


### How to find last value before 200

In [60]:
#converting description blocks into lists

desc_list = []
for i in descriptions:
    desc_list.append(re.findall(r"\w+,*", str(i)))

In [20]:
# Removing graduate courses

course_ids = course_ids[0:upper_div_i + 1]
course_names = course_names[0:upper_div_i + 1]
course_units = course_units[0:upper_div_i + 1]
desc_list = desc_list[0:upper_div_i + 1]

In [21]:
#extracting prerequisites.  

prereq_list = []

for i in np.arange(len(desc_list)):
    x = desc_list[i]
    try: 
        prereq_list.append(x[x.index('Prerequisites')+2:len(x)-1])
    except ValueError:
        print('prereq extraction error at index', i) 
        prereq_list.append('None')

## Check where the ValueErrors occur!  In some cases it may be faulty html e.g Math 157 has weird indent and misses prereq section

Before cell below, need to clear the following words/strings: 
'either'

and turn this:

'Students',
  'who',
  'have',
  'not',
  'completed',
  'listed',
  'prerequisites',
  'may',
  'enroll',
  'with',
  'consent',
  'of',
  'instructor'
  
into just " or + consent of instructor"


### might have to parse for or and make the lists 3d

In [22]:
prereqs_2d = []
for n in np.arange(len(course_ids)):
    ps = prereq_list[n]
    ps_2d = [[]]
    inner_i = 0
    for i in np.arange(len(ps)):    
        if ps[i] != 'and':
            ps_2d[inner_i].append(ps[i])
        #if ps[i] == 'students' or 
        else:
            ps_2d.append([])
            inner_i = inner_i + 1
    prereqs_2d.append(ps_2d)

In [ ]:
n = [['MATH', '140A', 'or', 'consent', 'of', 'instructor'], [asdasdasd]]

prereqs_3d = []
for n in np.arange(len(course_ids)):
    ps = prereq_list[n]:
        for i in np.arange(len(ps)):
            for i2 in ps[i]:
                if i2 

In [23]:
course_ids.index('100A')

21

In [24]:
prereqs_2d[45]

[['MATH', '140A', 'or', 'consent', 'of', 'instructor']]

all classes are going to have to be in the same object unless more looping

In [23]:
for prereq in prereqs_for_some_class:
    preprereqs = [] # prereqs for this prereq 
    look at prereqs of the prereq:
        repeat?


for i in prereqs:
    prereqs_3d[course_ids.index(i)]
    

SyntaxError: invalid syntax (<ipython-input-23-889b7b9f37a0>, line 3)

# Want to create dataframe with Columns: name, ID, prereqs, descs (?)

In [ ]:
course_info_df = pd.DataFrame({'Course id': course_ids, 'Course name': course_names, 'Course units' = course_units, 'Prereqs' = })

Course names array must be in 2d.

[[Math 2, Introduction to College Mathematics], [MATH 3, Precalculus] ...] 

In [78]:

#course_page = course_page[0:int(np.where(course_page == '<p class="course-name">MATH 200A-B-C. Algebra (4-4-4)</p>')[0])]

(205,)


In [51]:
courses = []
start = ' <p class="course-name"> '
stop = '</p>'
for i in course_page:
    for s in i:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-51-89e5bcb3f799>, line 5)

# GRAPHS

adjacency matrix or array of vertex pairs?

-adj lists

have to consider inter departmental case and how to access their respective course_ids etc

The loop should keep going as long as it can find the class in a prereq object.

dictionaries?  [Dept, ID] : [prereqs]

In [ ]:
def get_prereq(dept, num):
    a = course_ids.index(num) # [!] need to specify for dept
    return prereqs_3d[a]

#### The recursive part

input: course
intermediary input: the course's immediate prerequisites

the loop: should iterate down until no more prereq

output: A graph

In [ ]:
eg_prq = [ [['CSE', '8B'], ['CSE','11']],  [['MATH', '18'], ['MATH', '31AH']], [['MATH', '20E']], [['MATH','180A']] ]

map_graph = []
for i1 in eg_prq:
    for i2 in i1:
        prqs = get_prereq(i2[0], i2[1])
        
        
        
    map_graph.append([c1, c2])
        
        #dept = n[0][0]
                
                if eg_prq[i] == prereq_list[x] 

In [9]:
a = []
a.append(1)


In [10]:
a

[1]